<a href="https://colab.research.google.com/github/vickky0182/Personal/blob/main/Copy_of_amazonml_wtovec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
nltk.download('punkt')
import gensim
from gensim.models import Word2Vec,KeyedVectors
import gensim.downloader as api
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dim/train.csv')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Mounted at /content/gdrive


In [ ]:
""" Pre-processing
--A--
0. a) Find which all rows have null value for description and bullet points
   b) Find other row with same product_type_id and copy desc/bullet points
(convert to lowercase)
1. Remove Stop Words and Punctuations
2. Remove HTML Tags
3. number/100, dont remove them

--B--
0. Search for words similar to the word length or dimension

"""

' Pre-processing\n--A--\n0. a) Find which all rows have null value for description and bullet points\n   b) Find other row with same product_type_id and copy desc/bullet points\n(convert to lowercase)\n1. Remove Stop Words and Punctuations\n2. Remove HTML Tags\n3. number/100, dont remove them\n\n--B-- \n0. Search for words similar to the word length or dimension\n\n'

In [ ]:
! python -m nltk.downloader stopwords

/usr/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#2. Use BeautifulSoup to remove HTML tags
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
nltk.download('punkt')
nltk.download('stopwords')
def cleanify(trial_sentence):
  #Removing html tags
  soup = BeautifulSoup(trial_sentence, "html.parser")
  preprocessed_sentence = soup.get_text()
  #Convert to lower case
  words = word_tokenize(preprocessed_sentence.lower())
  #Removing stop words and punctuation
  stop_words = set(stopwords.words('english'))
  punctuations = set(string.punctuation)
  filtered_words = [word for word in words if word not in stop_words and word not in punctuations]
  preprocessed_sentence = " ".join(filtered_words)
  return preprocessed_sentence

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
print(cleanify("<p><strong> My name is the hero. i have, not been eating</p></strong>"))

name hero eating


In [ ]:
def w2vec(model,sentence):
  word_array = []
  for word in sentence:
    try:
      if model.similarity(word,'length')>0.3:
        word_array.append(model[word])
    except:
      continue
  desc_array = np.mean(word_array,axis=0)
  return desc_array

In [ ]:
model = api.load('word2vec-google-news-300')

In [ ]:
print(df.isnull().sum())

PRODUCT_ID               0
TITLE                   12
BULLET_POINTS       837364
DESCRIPTION        1157381
PRODUCT_TYPE_ID          0
PRODUCT_LENGTH           0
dtype: int64


In [ ]:
#replacing null values with 'Unknown'

df['DESCRIPTION'].fillna('Unknown',inplace=True)
df['BULLET_POINTS'].fillna('Unknown',inplace=True)

for i, row in df.iterrows():
    if pd.isna(row['TITLE']) and row['PRODUCT_TYPE_ID']:
        mask = (df['PRODUCT_TYPE_ID'] == row['PRODUCT_TYPE_ID']) & (~pd.isna(df['TITLE']))
        if mask.any():
          # Find the index of the first row that has the same PRODUCT_TYPE_ID and a non-empty title
          mask = (df['PRODUCT_TYPE_ID'] == row['PRODUCT_TYPE_ID']) & (~pd.isna(df['TITLE']))
          idx = df.loc[mask, 'TITLE'].sample(1, random_state=1).index[0]
          # Replace the missing value with the title of the selected row
        df.at[i, 'TITLE'] = df.at[idx, 'TITLE']




In [ ]:
print(df.isnull().sum())

PRODUCT_ID         0
TITLE              0
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
PRODUCT_LENGTH     0
dtype: int64


In [ ]:
# 1Correcting rows with null values
x_train = []
for index, row in df.iterrows():

  #print(index)
  pr_type_id = row.loc['PRODUCT_TYPE_ID']
  #print(pr_type_id)
  pr_id = row.loc['PRODUCT_ID']
  pr_length = row.loc['PRODUCT_LENGTH']
  #print(pr_length)
  new_df = df.loc[df['PRODUCT_TYPE_ID'] == pr_type_id]
  new_df = new_df[new_df['PRODUCT_ID']!=pr_id]
  t=new_df['PRODUCT_LENGTH'] - pr_length

  if pd.isnull(row.loc['DESCRIPTION']) and pd.isnull(row.loc['BULLET_POINTS']):
    #print(new_df)
    if len(t)>0:
      closest_row_index = t.abs().idxmin()
    else:
      continue
    #print(closest_row_index)
    row.loc['DESCRIPTION'] = new_df.loc[closest_row_index, 'DESCRIPTION']
    #print(row.loc['DESCRIPTION'])
    row.loc['BULLET_POINTS'] = new_df.loc[closest_row_index, 'BULLET_POINTS']
    #print(row.loc['BULLET_POINTS'])

  if pd.isnull(row.loc['DESCRIPTION']):
    #print(index)
    pr_type_id = row.loc['PRODUCT_TYPE_ID']
    #print(pr_type_id)
    pr_id = row.loc['PRODUCT_ID']
    pr_length = row.loc['PRODUCT_LENGTH']
    #print(pr_length)
    new_df = df.loc[df['PRODUCT_TYPE_ID'] == pr_type_id]
    new_df = new_df[new_df['PRODUCT_ID']!=pr_id]
    #print(new_df)
    if len(t)>0:
      closest_row_index = t.abs().idxmin()
    else:
      continue
    # print(closest_row_index)
    row.loc['DESCRIPTION'] = new_df.loc[closest_row_index, 'DESCRIPTION']
    # print(row.loc['DESCRIPTION'])

  elif pd.isnull(row.loc['BULLET_POINTS']):

    #print(index)
    pr_type_id = row.loc['PRODUCT_TYPE_ID']
    #print(pr_type_id)
    pr_id = row.loc['PRODUCT_ID']
    pr_length = row.loc['PRODUCT_LENGTH']
    #print(pr_length)
    new_df = df.loc[df['PRODUCT_TYPE_ID'] == pr_type_id]
    new_df = new_df[new_df['PRODUCT_ID']!=pr_id]
    #print(new_df)
    if len(t)>0:
      closest_row_index = t.abs().idxmin()
    else:
      continue
    #print(closest_row_index)
    row.loc['BULLET_POINTS'] = new_df.loc[closest_row_index, 'BULLET_POINTS']
    #print(row.loc['BULLET_POINTS'])

#Cleaning and word2vec
    # print(row.loc['DESCRIPTION'])
    # preprocessed_desc = cleanify(row.loc['DESCRIPTION'])
    # preprocessed_bullet = cleanify(row.loc['BULLET_POINTS'][0])
    # desc_array = w2vec(model,preprocessed_desc)
    # bullet_array = w2vec(model,preprocessed_bullet)
    # final_array = np.concatenate(([pr_type_id],desc_array, bullet_array))

    # x_train.append(final_array)


KeyboardInterrupt: ignored

In [ ]:
y_train = np.array(df['PRODUCT_LENGTH'])
x_train = np.array(df.loc[:, df.columns != "PRODUCT_LENGTH"])

In [ ]:
x_train.shape

(2249698, 5)

In [ ]:
for i in x_train:
  i[1]=cleanify(i[1])
  i[1]=w2vec(model,i[1])
  i[2]=cleanify(i[2])
  i[2]=w2vec(model,i[2])
  i[3]=cleanify(i[3])
  i[3]=w2vec(model,i[3])

# print(x_train[0])


/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-3-befee02acd21>:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(trial_sentence, "html.parser")
<ipython-input-3-befee02acd21>:11: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(trial_sentence, "html.parser")


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

model2 = Sequential()
model2.add(Dense(256, activation='relu', input_dim=5))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(1))
model2.compile(loss='mean_squared_error', optimizer='adam')
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               1536      
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 42,753
Trainable params: 42,753
Non-trainable params: 0
_________________________________________________________________


In [ ]:
[print(i.shape, i.dtype) for i in model2.inputs]
[print(o.shape, o.dtype) for o in model2.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model2.layers]

(None, 5) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>
dense_4 (None, 5) float32
dense_5 (None, 256) float32
dense_6 (None, 128) float32
dense_7 (None, 64) float32


[None, None, None, None]

In [ ]:
x_train[1]

array([2673191,
       "Marks & Spencer Girls' Pyjama Sets T86_2561C_Navy Mix_9-10Y",
       '[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% cotton (exclusive of trimmings),Ribbing - 98% cotton, 2% elastane]',
       'Unknown', 2755], dtype=object)

In [ ]:
train_X = x_train[0:20000]
train_Y = y_train[0:20000]
batch_size = 1000
runs = int(train_X.shape[0]/batch_size)
epochs =10
for e in range(0,epochs):
  print("Epoch:", e + 1, ".........................")
  for run in range (0,runs):
    print("Run:", run + 1)
    x_train = tf.convert_to_tensor(train_X[run*batch_size:(run+1)*batch_size].astype(np.float32))
    y_train = tf.convert_to_tensor(train_Y[run*batch_size:(run+1)*batch_size].astype(np.float32))
    model2.fit(x_train,y_train,epochs=1,batch_size=250)
    del x_train, y_train

Epoch: 1 .........................
Run: 1


ValueError: ignored